<a href="https://colab.research.google.com/github/usanovich/GI_Piragua/blob/main/Lluvia_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cuaderno para extracción de datos de precipitación a partir de la API del geoportal de Piragua-Corantioquia.**

In [ ]:
install.packages("urltools") # Instalación en sesión de librería para manejo de enlaces

Carga rutinaria de librerías empleadas en *script*.

In [ ]:
library(tidyverse)
library(urltools)
library(lubridate)
library(dplyr)
library(reticulate)
options(timeout=500)
options(scipen=999)

Para limpiar el entorno global.

In [ ]:
rm(list = ls())

Definimos parámetros (fechas) de inicio y final para extracción de datos.

In [ ]:
comenzar = "2020-12-01"
finalizar = "2020-12-17"

Leemos estaciones del catálogo de estaciones de Piragua.

In [ ]:
estaciones = read.csv("http://www.piraguacorantioquia.com.co/redes_nuevo/images/Estaciones_Piragua.csv")

Aplicamos filtro en catálogo por "Pluviógrafo" usando función ***filter()***; podríamos aplicar filtros adicionales, por ejemplo: 


```
filter(tipo = "Pluviógrafo, territorial = "Aburrá Sur")
```
La anterior línea permite filtrar por las estaciones territoriales ubicadas en el Aburrá Sur.


In [ ]:
estaciones_pluvio = estaciones %>%
  filter(tipo == "Pluviógrafo")

Creamos categorías únicas para municipios, esto con el fin de discernir entre varias estaciones que se encuentran en un mismo municipio.

In [ ]:
estaciones_pluvio$municipio = make.names(estaciones_pluvio[,4], unique = TRUE)

Creamos una función llamada ***get_data()*** para extracción de datos desde la API del geoportal de Piragua.

In [ ]:
get_data = function(estacion, inicio, final){
  url="https://www.piraguacorantioquia.com.co/api/precipitacion/40?date_estacion__gte=2020-11-17&date_estacion__lt=2020-11-23&downloadfile"
  url2 = param_set(url, "date_estacion__gte", inicio)
  url3 = param_set(url2, "date_estacion__lt", final)
  data = read.csv(gsub("40", estacion, url3))
  data = na.omit(data)
  data = data[!(data$fechas == ""), ] # Para borrar campos de fechas vacíos
  data$fechas = as.POSIXct(data$fechas, "%Y-%m-%d %H-%M")
  data$fechas = floor_date(data$fechas, "5 mins")
  data2 = data %>%
    group_by(fechas) %>%
    dplyr::summarise(muestra = mean(muestra))
}

Función de extracción y conversión de datos.

In [ ]:
get_lluvia = function() {
  
  lluvia = list()
  
  # Guardamos en lista de listas todos los datasets de las estaciones 
  # para una territorial en particular
  for (i in 1:nrow(estaciones_pluvio)) {
    
    estacion = estaciones_pluvio$codigo[i]
    lluvia[[i]] = get_data(estacion, comenzar, finalizar)
    
  }
  # Agregamos una columna de municipios a cada dataframe de la lista
  for (i in 1:length(lluvia)) {
    
    if (nrow(lluvia[[i]]) > 0) {
      lluvia[[i]] = data.frame(lluvia[[i]],
                               territorial = estaciones_pluvio[i,5],
                               municipios = estaciones_pluvio[i,4])
    }
  }
  #Para eliminar listas con 0 filas
  lluvia = lluvia[sapply(lluvia, nrow) > 0]
  #Creamos un dataframe global con la lista de estaciones que tienen datos
  lluvia = do.call(rbind, lluvia)
  
}

Extraemos, ahora si, los datos iterando sobre cada estación pluviográfica.

In [ ]:
lluvia = get_lluvia()

Agrupamos por municipios y hallamos su acumulado para la temporalidad dada.




In [ ]:
lluvia2 = lluvia %>%
  group_by(municipios, territorial) %>%
  dplyr::summarise(muestra = sum(muestra))

`summarise()` regrouping output by 'municipios' (override with `.groups` argument)



In [ ]:
write.table(lluvia2,"lluvia2.txt", sep = ";", 
            col.names = c("municipios", "territorial", "muestra"),
            row.names = F)